# __텍스트 분류 모델 만들기__

## __0. 데이터 준비하기__

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN={발급받은 개인 토큰}

In [2]:
%%thanosql
COPY movie_review_train FROM "tutorial_data/movie_review_data/movie_review_train.csv"

Success


In [3]:
%%thanosql
COPY movie_review_test FROM "tutorial_data/movie_review_data/movie_review_test.csv"

Success


## __1. 데이터 세트 확인__

영화 리뷰 감정 분류 모델을 만들기 위해 우리는 ThanoSQL DB에 저장되어 있는 movie_review_train 테이블을 사용합니다. 아래의 쿼리문을 실행하여 테이블 내용을 확인합니다.

In [4]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


## __2. 사전 학습된 모델을 사용하여 영화 리뷰 감정 분류 결과 예측__

먼저 저희가 사전에 학습해둔 모델로 바로 결과를 예측해보겠습니다. 다음 쿼리문을 실행하면 tutorial_text_classification 모델을 사용하여 영화 리뷰 분류 결과를 예측해볼 수 있습니다.

In [5]:
%%thanosql
PREDICT USING tutorial_text_classification
OPTIONS (
    text_col='review',
    label_col='sentiment'
    )
AS
SELECT *
FROM movie_review_test

,review,sentiment,predicted
0,"I read the book before seeing the movie, and t...",positive,positive
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,positive
2,"Yesterday I attended the world premiere of ""De...",positive,positive
3,Moonwalker is a Fantasy Music film staring Mic...,positive,positive
4,"Welcome to Oakland, where the dead come out to...",positive,positive
...,...,...,...
995,I remember catching this movie on one of the S...,negative,negative
996,CyberTracker is set in Los Angeles sometime in...,negative,negative
997,"There is so much that is wrong with this film,...",negative,negative
998,"I am a firm believer that a film, TV serial or...",positive,positive


## __3. 텍스트 분류 모델 만들기__

이전 단계에서 확인한 movie_review_train 데이터 세트를 사용하여 텍스트 분류 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 my_movie_review_classifier라는 이름의 모델을 만듭니다.

In [6]:
%%thanosql
BUILD MODEL my_movie_review_classifier
USING ElectraEn
OPTIONS (
    text_col='review',
    label_col='sentiment',
    epochs=1,
    batch_size=4
    )
AS
SELECT *
FROM movie_review_train

Success


## __4. 생성된 모델을 사용하여 영화 리뷰 감정 분류 결과 예측__

이전 단계에서 만든 텍스트 분류 예측 모델을 사용해서 특정 리뷰(학습에 이용되지 않은 데이터 테이블, movie_review_test)의 목표값을 예측해 봅니다.

In [7]:
%%thanosql
PREDICT USING my_movie_review_classifier
OPTIONS (
    text_col='review'
    )
AS
SELECT *
FROM movie_review_test

,review,sentiment,predicted
0,"I read the book before seeing the movie, and t...",positive,positive
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,positive
2,"Yesterday I attended the world premiere of ""De...",positive,positive
3,Moonwalker is a Fantasy Music film staring Mic...,positive,positive
4,"Welcome to Oakland, where the dead come out to...",positive,negative
...,...,...,...
995,I remember catching this movie on one of the S...,negative,negative
996,CyberTracker is set in Los Angeles sometime in...,negative,negative
997,"There is so much that is wrong with this film,...",negative,negative
998,"I am a firm believer that a film, TV serial or...",positive,positive
